In [3]:
!pip install torch
!pip install transformers
!pip install SpeechRecognition
!pip install vaderSentiment
!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.2 MB/s eta 0:00:00


In [7]:
import torch
import gdown
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import speech_recognition as sr
from pydub import AudioSegment

# Step 1: Convert Audio to Text (same as before)
def convert_audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export("temp.wav", format="wav")

    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)

    try:
        conversation_text = recognizer.recognize_google(audio_data)
        return conversation_text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Step 2: Text Preprocessing (same as before)
def preprocess_text(text):
    sentences = text.split(".")
    return sentences

# Step 3: Deep Learning Sentiment Analysis
def analyze_sentiment(sentences):
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"  # Pre-trained BERT model for sentiment analysis
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    positive, negative, neutral = 0, 0, 0

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=-1)
        sentiment_score = torch.argmax(probs)

        # Mapping sentiment_score to positive, negative, neutral
        if sentiment_score in [4, 3]:  # 4 and 3 represent strong and weak positive respectively
            positive += 1
        elif sentiment_score in [0, 1]:  # 0 and 1 represent strong and weak negative respectively
            negative += 1
        else:  # 2 represents neutral
            neutral += 1

    return positive, negative, neutral

# Step 4: Percentage Calculation (same as before)
def calculate_percentage(positive, negative, neutral):
    total = positive + negative + neutral
    if total == 0:
        return 0, 0, 0
    positive_percentage = (positive / total) * 100
    negative_percentage = (negative / total) * 100
    neutral_percentage = (neutral / total) * 100
    return positive_percentage, negative_percentage, neutral_percentage

# Main function to run all steps
def main(audio_file):
    print("Converting audio to text...")
    conversation_text = convert_audio_to_text(audio_file)
    if not conversation_text:
        print("No text extracted from audio.")
        return

    print("Preprocessing text...")
    sentences = preprocess_text(conversation_text)

    print("Analyzing sentiment with BERT...")
    positive, negative, neutral = analyze_sentiment(sentences)

    print("Calculating percentages...")
    positive_percentage, negative_percentage, neutral_percentage = calculate_percentage(positive, negative, neutral)

    print(f"Positive Sentiment: {positive_percentage:.2f}%")
    print(f"Negative Sentiment: {negative_percentage:.2f}%")
    print(f"Neutral Sentiment: {neutral_percentage:.2f}%")

     # Assessing customer satisfaction
    if positive_percentage > negative_percentage and positive_percentage > neutral_percentage:
        print("The call is classified as Positive in terms of customer satisfaction.")
    elif negative_percentage > positive_percentage and negative_percentage > neutral_percentage:
        print("The call is classified as Negative in terms of customer satisfaction.")
    else:
        print("The call is classified as Neutral in terms of customer satisfaction.")

if __name__ == "__main__":
    # Download file from Google Drive (update the URL with your file's ID)
    url = "https://drive.google.com/uc?id=1siHFN6GdFIFs9oaqou3sZ__UFtaCq7i7"
    output = "AudioCall.mp3"
    gdown.download(url, output, quiet=False)

    # Use the downloaded file
    audio_file_path = "AudioCall.mp3"
    main(audio_file_path)

Downloading...
From: https://drive.google.com/uc?id=1siHFN6GdFIFs9oaqou3sZ__UFtaCq7i7
To: /content/AudioCall.mp3
100%|██████████| 3.75M/3.75M [00:00<00:00, 222MB/s]


Converting audio to text...
Preprocessing text...
Analyzing sentiment with BERT...
Calculating percentages...
Positive Sentiment: 100.00%
Negative Sentiment: 0.00%
Neutral Sentiment: 0.00%
The call is classified as Positive in terms of customer satisfaction.
